# 잠재 디리클레 할당(Latent Dirichlet Allocation, LDA)
---

- 토픽의 개수를 직접 정해줘야한다. (하이퍼파라미터)
- 토픽의 제목이 정해지진 않으며 사용자가 판단해야 함.
- BoW 기반 DTM이나 TF-IDF를 사용하기 때문에, LDA역시 단어의 순서는 신경쓰지 않는다.
- LDA의 가정
  1. 문서에 사용할 N개의 단어를 정한다.
  2. 문서에 사용할 토픽의 혼합을 확률 분포에 기반해서 결정
  3. 문서에 사용할 각 단어를 정한다.
- LDA 수행
  1. 토픽의 개수 k를 정한다.
  2. 모든 단어를 k개 중 하나의 토픽에 할당한다.
  3. 단어를 토픽에 할당할 때는 두 가지를 살펴본다.
    - 해당 문서의 토픽 비율이 어떻게 분포 되어있는지
    - 다른 문서의 해당 단어가 어떤 토픽으로 할당되어있는지
- LSA와 LDA의 차이
  - LSA : DTM을 차원 축소하여 축소 차원에서 근접 단어들을 토픽으로 묶어준다.
  - LDA : 단어가 특정 토픽에 존재할 확률과 문서에 특정 토픽이 존재할 확률을 결합 확률로 추정하여 토픽을 추출
   

# LDA 실습
---

In [0]:
!pip install konlpy

In [0]:
room = ["바닥은 삐걱거리는 나무판자로 되어있다.",\
        "바닥은 차갑다",\
        "방 가운데에는 돌로 된 탁자가 하나 놓여있다.",\
        "방에 문이 하나 있다.",\
        "문은 나무로 되어있다.",\
        "문은 열리지 않는다.",\
        "문에는 잠금장치가 걸려있다.",\
        "방에 침대가 하나 있다.",\
        "탁자 옆에는 의자가 두 개 놓여있다.",\
        "침대 밑에는 열쇠가 있다.",\
        "방에 창문이 없다.",\
        "방 안에 또 다른 방이 있다.",\
        "방과 방이 창문으로 연결되어 있다."]

In [0]:
from konlpy.tag import Kkma
import re
kma = Kkma()
tk_list=[re.sub("(\.)","",sen) for sen in room]
tk_list = [kma.nouns(sen) for sen in tk_list] # 명사만 토큰화. 해당 예시 문서들에서는 명사 토큰화로 불용어가 거의 제거됨.

In [19]:
## 정수 인코딩과 함께 빈도수 기록 -> 단어 집합 만들기
from gensim import corpora
dic = corpora.Dictionary(tk_list)
corpus = [dic.doc2bow(text) for text in tk_list]
print(corpus[0])

[(0, 1), (1, 1), (2, 1), (3, 1)]


In [20]:
print(dic[2]) # 바닥은 2로 정수 인코딩 되었으며, 첫번째 문서에서 빈도수가 1이다

바닥


In [21]:
len(dic) # 총 23개의 단어

23

In [22]:
## LDA 모델 훈련시키기
import gensim
NUM_TOPICS = 4 # 4개의 토픽
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dic, passes=15)
topics = ldamodel.print_topics(num_words=4) # 토픽당 4개의 단어만 출력
for t in topics:
  print(t)

(0, '0.271*"문" + 0.107*"잠금장치" + 0.107*"장치" + 0.107*"잠금"')
(1, '0.108*"탁자" + 0.108*"나무" + 0.107*"의자" + 0.107*"옆"')
(2, '0.163*"방" + 0.125*"하나" + 0.087*"침대" + 0.087*"바닥"')
(3, '0.193*"방" + 0.191*"창문" + 0.106*"연결" + 0.106*"방과"')


In [23]:
total_topics = ldamodel.print_topics() # 10개 단어 출력
for t in total_topics:
  print(t)

(0, '0.271*"문" + 0.107*"잠금장치" + 0.107*"장치" + 0.107*"잠금" + 0.022*"하나" + 0.022*"방" + 0.021*"나무" + 0.021*"바닥" + 0.021*"안" + 0.021*"열쇠"')
(1, '0.108*"탁자" + 0.108*"나무" + 0.107*"의자" + 0.107*"옆" + 0.107*"개" + 0.099*"문" + 0.021*"바닥" + 0.021*"방" + 0.021*"안" + 0.021*"하나"')
(2, '0.163*"방" + 0.125*"하나" + 0.087*"침대" + 0.087*"바닥" + 0.048*"돌" + 0.048*"가운데" + 0.048*"나무판자" + 0.048*"판자" + 0.048*"밑" + 0.048*"열쇠"')
(3, '0.193*"방" + 0.191*"창문" + 0.106*"연결" + 0.106*"방과" + 0.021*"문" + 0.021*"바닥" + 0.021*"안" + 0.021*"나무" + 0.021*"하나" + 0.021*"열쇠"')


In [0]:
## LDA 모델 시각화
!pip install pyLDAvis

In [25]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel,corpus,dic)
pyLDAvis.display(vis)

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [26]:
## 토픽 비율 확인
for i,topic_list in enumerate(ldamodel[corpus]):
  print("text : ",tk_list[i])
  print("topic rate : ",topic_list)
  print("--------------------------")

text :  ['바닥', '나무', '나무판자', '판자']
topic rate :  [(0, 0.050113596), (1, 0.05227123), (2, 0.8475029), (3, 0.05011227)]
--------------------------
text :  ['바닥']
topic rate :  [(0, 0.12517415), (1, 0.12517431), (2, 0.624479), (3, 0.1251726)]
--------------------------
text :  ['방', '가운데', '돌', '탁자', '하나']
topic rate :  [(0, 0.04174286), (1, 0.04313004), (2, 0.872796), (3, 0.042331137)]
--------------------------
text :  ['방', '문', '하나']
topic rate :  [(0, 0.30470264), (1, 0.06386232), (2, 0.56627065), (3, 0.06516442)]
--------------------------
text :  ['문', '나무']
topic rate :  [(0, 0.26418215), (1, 0.5670486), (2, 0.08532665), (3, 0.08344257)]
--------------------------
text :  ['문']
topic rate :  [(0, 0.62214977), (1, 0.12765454), (2, 0.12514567), (3, 0.12505001)]
--------------------------
text :  ['문', '잠금', '잠금장치', '장치']
topic rate :  [(0, 0.84966856), (1, 0.05025348), (2, 0.050031137), (3, 0.050046764)]
--------------------------
text :  ['방', '침대', '하나']
topic rate :  [(0, 0.06255